In [1]:
import os
import numpy as np
import xarray as xr

from glob import glob
from functools import partial
from os import makedirs as mkdir
from multiprocessing import get_context
from datetime import datetime, timedelta

os.environ['OMP_NUM_THREAD'] = '1'

In [2]:
nlat, xlat = 40, 45
nlon, xlon = -115, -110

In [3]:
def download_grib(url, subset_str, tmp):
    from subprocess import Popen
    # import requests
    
    filename = url.split('file=')[1].split('&')[0]
    filename = filename.replace('.co.', '.%s.'%subset_str)
        
    if not os.path.isfile(tmp + filename):
        print('Downloading %s'%filename)
        
        # r = requests.get(url, allow_redirects=True)
        # open(tmp + filename, 'wb').write(r.content)
        
        cmd = 'wget -O "%s" "%s"'%(tmp + filename, url)
        Popen(cmd, shell=True)
    
    return filename

In [4]:
def repack_nbm_grib2(f):
    import pygrib
    import gc
    
    print(f.split('/')[-1])
    
    try:
        grb = pygrib.open(f)
        msgs = grb.read()
                
        init = str(msgs[0]).split(':')[-2].split(' ')[-1]
        init = datetime.strptime(init, '%Y%m%d%H%M')
        
        fhr = msgs[0]['endStep']
        valid = np.datetime64(init + timedelta(hours=fhr))
        
        lons, lats = msgs[0].data()[2], msgs[0].data()[1]
        
    except:
        raise
        return None
    
    else:
        probability, probability_labels = [], []
        percentile, percentile_labels = [], []

        deterministic, deterministic_labels = [], []
        got_deterministic = {i:False for i in [1, 6, 12, 24]}

        for msg in msgs:

            interval = msg['stepRange'].split('-')
            interval = int(interval[1]) - int(interval[0])

            if 'Probability of event' in str(msg):    
                    # Probability of event above upper limit (> 0.254) NOT inclusive
                    threshold = round(msg['upperLimit']/25.4, 2)
                    probability.append([msg.values])
                    probability_labels.append([interval, threshold])

            elif 'percentileValue' in msg.keys():                
                percentile.append([msg.values])
                percentile_labels.append([interval, msg['percentileValue']])

            else:
                if got_deterministic[interval] == False:
                    deterministic_labels.append(interval)
                    deterministic.append(msg.values)
                    got_deterministic[interval] = True
                else:
                    pass

        grb.close()
        gc.collect()
        
        deterministic_labels = np.array(deterministic_labels)
        deterministic_labels = deterministic_labels[np.argsort(deterministic_labels)]
        deterministic = np.array(deterministic)[np.argsort(deterministic_labels)]

        probability = np.array(probability, dtype=object).reshape(-1, lats.shape[0], lats.shape[1])
        probability_labels = np.array(probability_labels)

        percentile = np.array(percentile, dtype=object).reshape(-1, 99, lats.shape[0], lats.shape[1])
        percentile_labels = np.array(percentile_labels)

        deterministic = xr.DataArray(deterministic.astype(np.float32), name='pop',
                        dims=('interval', 'y', 'x'),
                        coords={'interval':('interval', deterministic_labels),
                                'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

        pop = xr.DataArray(probability[:3].astype(np.float32), name='pop',
                        dims=('interval', 'y', 'x'),
                        coords={'interval':('interval', probability_labels[:3, 0]),
                                'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

        probability = xr.DataArray([probability[2:].astype(np.float32)], name='probability',
                        dims=('interval', 'threshold', 'y', 'x'),
                        coords={'interval':('interval', [24]), 'threshold':('threshold', probability_labels[2:,1]),
                                'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

        percentile = xr.DataArray(percentile.astype(np.float32), name='percentile',
                        dims=('interval', 'percentile', 'y', 'x'),
                        coords={'interval':('interval', np.unique(percentile_labels[:, 0])), 'percentile':('percentile', range(1, 100)),
                                'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

        ds = xr.Dataset()
        
        # ds['fhr'] = fhr
        ds['time'] = valid
        ds.attrs['InitTime'] = str(init)
        
        ds['qpf'] = deterministic
        ds['pop'] = pop
        ds['probx'] = probability
        ds['pqpf'] = percentile

        return ds

In [5]:
# Set up init to use sys.argv later
init = datetime(2020, 10, 7, 12)
yyyy, mm, dd, hh = init.year, init.month, init.day, init.hour

In [6]:
base = 'https://nomads.ncep.noaa.gov/cgi-bin/filter_blend.pl?'
var = '&var_APCP=on'
region = '&subregion=&leftlon={:.2f}&rightlon={:.2f}&toplat={:.2f}&bottomlat={:.2f}'.format(nlon, xlon, xlat, nlat)
mdir = '&dir=%2Fblend.{:04d}{:02d}{:02d}%2F{:02d}%2Fqmd'.format(yyyy, mm, dd, hh)

url_list = []

# Need to fix the data processing below to allow for sub24 leads
for fhr in np.arange(24, 180+1, 6):
    file = 'file=blend.t{:02d}z.qmd.f{:03d}.co.grib2'.format(hh, fhr)
    url_list.append(base + file + var + region + mdir)

In [7]:
tmpdir = '/scratch/general/lustre/u1070830/nbm/tmp/'; mkdir(tmpdir, exist_ok=True)
datadir = '/scratch/general/lustre/u1070830/nbm/'; mkdir(datadir, exist_ok=True)

In [8]:
download_grib_mp = partial(download_grib, subset_str='WR', tmp=tmpdir)

if __name__ == '__main__':
    
    with get_context('fork').Pool(4) as p:
        flist = p.imap_unordered(download_grib_mp, url_list, chunksize=1)
        p.close()
        p.join()

flist = sorted(flist)

In [9]:
filelist = sorted(glob(tmpdir + '*.grib2'))

for i, f in enumerate(flist):
    print(i, f)

0 blend.t12z.qmd.f024.WR.grib2
1 blend.t12z.qmd.f030.WR.grib2
2 blend.t12z.qmd.f036.WR.grib2
3 blend.t12z.qmd.f042.WR.grib2
4 blend.t12z.qmd.f048.WR.grib2
5 blend.t12z.qmd.f054.WR.grib2
6 blend.t12z.qmd.f060.WR.grib2
7 blend.t12z.qmd.f066.WR.grib2
8 blend.t12z.qmd.f072.WR.grib2
9 blend.t12z.qmd.f078.WR.grib2
10 blend.t12z.qmd.f084.WR.grib2
11 blend.t12z.qmd.f090.WR.grib2
12 blend.t12z.qmd.f096.WR.grib2
13 blend.t12z.qmd.f102.WR.grib2
14 blend.t12z.qmd.f108.WR.grib2
15 blend.t12z.qmd.f114.WR.grib2
16 blend.t12z.qmd.f120.WR.grib2
17 blend.t12z.qmd.f126.WR.grib2
18 blend.t12z.qmd.f132.WR.grib2
19 blend.t12z.qmd.f138.WR.grib2
20 blend.t12z.qmd.f144.WR.grib2
21 blend.t12z.qmd.f150.WR.grib2
22 blend.t12z.qmd.f156.WR.grib2
23 blend.t12z.qmd.f162.WR.grib2
24 blend.t12z.qmd.f168.WR.grib2
25 blend.t12z.qmd.f174.WR.grib2
26 blend.t12z.qmd.f180.WR.grib2


In [14]:
# if __name__ == '__main__':
    
#     with get_context('fork').Pool(4) as p:
#         output = p.imap_unordered(repack_nbm_grib2, filelist, chunksize=1)
#         p.close()
#         p.join()

output = [repack_nbm_grib2(f) for f in filelist]
output = xr.concat([i for i in output if i is not None], dim='time')
output

<xarray.Dataset>
Dimensions:     (interval: 4, percentile: 99, threshold: 10, time: 27, x: 198, y: 250)
Coordinates:
  * interval    (interval) int64 1 6 12 24
  * threshold   (threshold) float64 0.01 0.1 0.25 0.5 1.0 2.0 3.0 4.0 5.0 6.0
  * time        (time) datetime64[ns] 2020-10-07T12:00:00 ... 2020-10-14
    lats        (y, x) float64 39.53 39.53 39.53 39.54 ... 45.43 45.44 45.44
    lons        (y, x) float64 -114.9 -114.9 -114.9 ... -110.2 -110.1 -110.1
  * percentile  (percentile) int64 1 2 3 4 5 6 7 8 9 ... 92 93 94 95 96 97 98 99
Dimensions without coordinates: x, y
Data variables:
    qpf         (time, interval, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    pop         (time, interval, y, x) float32 nan nan nan ... 38.0 39.0 39.0
    probx       (time, interval, threshold, y, x) float64 nan nan ... 0.0 0.0
    pqpf        (time, interval, percentile, y, x) float32 nan nan ... 14.314
Attributes:
    InitTime:  2020-10-06 12:00:00

In [17]:
compress = {'compression':'gzip', 'compression_opts':9}
encoding = {var:compress for var in output.data_vars if var != 'time'}
output.to_netcdf(tmpdir + './test_output.nc', engine='h5netcdf', encoding=encoding)